In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
import pickle

# Load the dataset
data = pd.read_csv(r'P:\project ides for final ptoject\project\IDS_Federated learning model\model\reduced_dataset.csv')

# Check if the dataset loaded correctly
print("Initial Data:")
print(data.head())  # Check the first few rows
print(data.info())  # Check the summary of the DataFrame

# Select relevant columns
selected_columns = [
    'duration', 'protocol', 'service', 'src_bytes', 'dst_bytes',
    'src_pkts', 'dst_pkts', 'src_ip_bytes', 'dst_ip_bytes',
    'conn_state', 'attack'
]
data = data[selected_columns]

# Drop rows with NaN values
data.dropna(inplace=True)

# Check how many samples are left after dropping NaN
print(f'Number of samples after dropping NaN: {len(data)}')

# Filter the dataset to include both 'normal' and 'attack' labels
data = data[data['attack'].isin(['normal', 'attack'])]

# Encode labels as binary (normal = 0, attack = 1)
data['attack'] = data['attack'].map({'normal': 0, 'attack': 1})

# Split the dataset into features and target variable
X = data.drop('attack', axis=1)
y = data['attack'].values

# Check if X and y are empty before splitting
print(f'X shape: {X.shape}, y shape: {y.shape}')  # Print the shapes

# Define categorical and numerical columns
categorical_features = ['protocol', 'service', 'conn_state']
numerical_features = ['duration', 'src_bytes', 'dst_bytes', 'src_pkts', 'dst_pkts', 'src_ip_bytes', 'dst_ip_bytes']

# Create a preprocessor for the data
preprocessor = ColumnTransformer(
    transformers=[ 
        ('num', StandardScaler(), numerical_features),  # Scale numerical features
        ('cat', OneHotEncoder(), categorical_features)  # One-hot encode categorical features
    ])

# Create a pipeline with preprocessor and TruncatedSVD
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svd', TruncatedSVD(n_components=10))  # Adjust based on your needs
])

# Transform the features
X_transformed = pipeline.fit_transform(X)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Define hyperparameters
epochs = 1000
batch_size = 32
learning_rate = 0.01

# Define the sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the derivative of the sigmoid function
def sigmoid_derivative(x):
    return x * (1 - x)

# Define the training function
def train_model(X_train, y_train, layers, epochs=1000, learning_rate=0.01, batch_size=32):
    # Initialize weights and biases
    weights = []
    biases = []
    input_size = X_train.shape[1]
    
    # Create weight and bias matrices
    for i in range(len(layers) - 1):
        weight_matrix = np.random.rand(layers[i], layers[i + 1]) * 0.01
        bias_vector = np.zeros((1, layers[i + 1]))
        weights.append(weight_matrix)
        biases.append(bias_vector)

    # Training loop
    for epoch in range(epochs):
        for X_batch, y_batch in get_mini_batches(X_train, y_train, batch_size):
            # Forward propagation
            activations = [X_batch]
            for w, b in zip(weights, biases):
                z = np.dot(activations[-1], w) + b
                a = sigmoid(z)
                activations.append(a)

            # Backward propagation
            output_layer = activations[-1]
            error = y_batch.reshape(-1, 1) - output_layer
            deltas = [error * sigmoid_derivative(output_layer)]

            for i in reversed(range(len(weights) - 1)):
                deltas.append(deltas[-1].dot(weights[i + 1].T) * sigmoid_derivative(activations[i + 1]))

            deltas.reverse()

            # Update weights and biases
            for i in range(len(weights)):
                weights[i] += activations[i].T.dot(deltas[i]) * learning_rate
                biases[i] += np.sum(deltas[i], axis=0, keepdims=True) * learning_rate

        # Print loss every 100 epochs
        if epoch % 100 == 0:
            predictions = activations[-1]
            loss = -np.mean(y_batch * np.log(predictions + 1e-15) + (1 - y_batch) * np.log(1 - predictions + 1e-15))
            print(f'Epoch {epoch}, Loss: {loss:.4f}')

    # Save weights and biases as NumPy arrays in a pickle file
    model_data = {'weights': weights, 'biases': biases}
    with open('model_weights_and_biases3.pickle', 'wb') as f:
        pickle.dump(model_data, f)

    print("Model weights and biases have been saved to 'model_weights_and_biases3.pickle'.")

    return weights, biases

# Function to create mini-batches
def get_mini_batches(X, y, batch_size=32):
    n_samples = X.shape[0]
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        batch_indices = indices[start:end]
        yield X[batch_indices], y[batch_indices]

# Define layer structure
layers = [X_train.shape[1], 32, 16, 1]  # Input layer, hidden layers, output layer

# Train the model
weights, biases = train_model(X_train, y_train, layers, epochs=epochs, learning_rate=learning_rate, batch_size=batch_size)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

# Function to perform forward propagation using the trained weights and biases
def forward_propagation(X, weights, biases):
    activations = [X]
    for w, b in zip(weights, biases):
        z = np.dot(activations[-1], w) + b
        a = sigmoid(z)
        activations.append(a)
    return activations[-1]

# Predict the outputs for the validation set
y_val_pred = forward_propagation(X_val, weights, biases)

# Convert predictions to binary labels (0 or 1)
y_val_pred_binary = (y_val_pred > 0.5).astype(int)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_val, y_val_pred_binary)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap='Blues', interpolation='nearest')
plt.title("Confusion Matrix")
plt.colorbar()
plt.xticks([0, 1], ['Normal', 'Attack'])
plt.yticks([0, 1], ['Normal', 'Attack'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, conf_matrix[i, j], ha='center', va='center', color='red')
plt.show()

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_val, y_val_pred_binary)
precision = precision_score(y_val, y_val_pred_binary)
recall = recall_score(y_val, y_val_pred_binary)
f1 = f1_score(y_val, y_val_pred_binary)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Print classification report for detailed metrics
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred_binary, target_names=['Normal', 'Attack']))
